In [1]:
#BiocManager::install("edgeR")
library(edgeR)

Loading required package: limma


In [2]:
dir_work = '/Users/kef74yk/Dropbox_p/data/04_Convergence_Duplication/20190429_tmm_normalization/'
setwd(dir_work)
date_orthofinder = 'Mar22'
date_sra = '2018_5_1'
dir_ensembl = '/Users/kef74yk/Dropbox_w/db/Ensembl/release-91/'
dir_count = paste0(dir_ensembl, 'kallisto_summary/', date_sra, '/est_counts.masked.kallisto.tsv/')
dir_efflen = paste0(dir_ensembl, 'kallisto_summary/', date_sra, '/eff_length.masked.kallisto.tsv/')
dir_id = paste0(dir_ensembl, 'id_mapping/')
dir_tmm_logfpkm = paste0(dir_ensembl, 'kallisto_summary/', date_sra, '/fpkm.tmm.kallisto.gene.log.tsv/')
dir_orthofinder = paste0(dir_ensembl, 'orthofinder/')
file_singlecopy = paste0(dir_orthofinder, 'Results_', date_orthofinder, '/SingleCopyOrthogroups.txt')
file_orthogroup = paste0(dir_orthofinder, 'Results_', date_orthofinder, '/Orthogroups.csv')
single_orthogroups = read.table(file_singlecopy, header=FALSE)$V1
df_og = read.table(file_orthogroup, header=TRUE, sep='\t', row.names=1)
df_singleog = df_og[(rownames(df_og) %in% single_orthogroups),]
for (col in 1:ncol(df_singleog)) {
    df_singleog[,col] = sub('.*_', '', df_singleog[,col])
}
df_og = NULL
spp_filled = colnames(df_singleog)
spp = sub('_', ' ', spp_filled)
num_selected_spp = length(spp)
cat('num selected species:', num_selected_spp, '\n')
cat('selected species:', spp, '\n')

num selected species: 21 
selected species: Anolis carolinensis Astyanax mexicanus Bos taurus Callithrix jacchus Canis lupus Chinchilla lanigera Danio rerio Gadus morhua Gallus gallus Homo sapiens Macaca mulatta Monodelphis domestica Mus musculus Oreochromis niloticus Ornithorhynchus anatinus Oryctolagus cuniculus Oryzias latipes Ovis aries Rattus norvegicus Sus scrofa Xenopus tropicalis 


In [3]:
transcript2gene_transformation = function(tc, idfile) {
    sp_mapping = read.table(idfile, quote="", sep="\t", header=TRUE, stringsAsFactors=FALSE)
    tc$ensembl_transcript_id = sub("\\..*", "", rownames(tc))
    tc = merge(tc, sp_mapping[,c("Gene.stable.ID","Transcript.stable.ID")], by.x="ensembl_transcript_id", by.y="Transcript.stable.ID", sort=FALSE)
    tc = tc[,-1]
    tc = aggregate(x=tc[,-ncol(tc)], by=list(tc$Gene.stable.ID), FUN=sum)
    rownames(tc) = tc$Group.1
    tc = tc[,-1]
    return(tc)
}

In [4]:
#spp_filled = c('Anolis_carolinensis','Monodelphis_domestica','Chinchilla_lanigera')

uncorrected = list()
df_sog = df_singleog
files = list.files(dir_count)
for (sp in spp_filled) {
    infile = paste0(sp, '.est_counts.masked.kallisto.tsv')
    infile_path = paste0(dir_count, infile)
    if (file.exists(infile_path)) {
        cat('Input file found, reading:', infile, '\n')
        dat = read.table(infile_path, header=TRUE, row.names=1, sep='\t')
        dat = dat[,(colnames(dat)!='length')]
        genus_cap = tolower(substring(sp,1,1))
        spec_name = strsplit(sp,'_')[[1]][2]
        idfile = paste0(dir_id, genus_cap, spec_name, '_gene_ensembl.tsv')
        dat = transcript2gene_transformation(dat, idfile)
        colnames(dat) = paste(sp, colnames(dat), sep='_')
        uncorrected[[sp]] = dat
        #rownames(dat) = sub('\\..*','', rownames(dat))
        df_sog = merge(df_sog, dat, by.x=sp, by.y="row.names", all.x=TRUE, all.y=FALSE, sort=FALSE)
    } else {
        cat('Input file not found:', infile, '\n')
    }
}
df_sog = df_sog[,-(1:num_selected_spp)]
df_sog = apply(df_sog, 2, as.integer)
rownames(df_sog) = rownames(df_singleog)
head(df_sog)

Input file found, reading: Anolis_carolinensis.est_counts.masked.kallisto.tsv 
Input file found, reading: Astyanax_mexicanus.est_counts.masked.kallisto.tsv 
Input file found, reading: Bos_taurus.est_counts.masked.kallisto.tsv 
Input file found, reading: Callithrix_jacchus.est_counts.masked.kallisto.tsv 
Input file found, reading: Canis_lupus.est_counts.masked.kallisto.tsv 
Input file found, reading: Chinchilla_lanigera.est_counts.masked.kallisto.tsv 
Input file found, reading: Danio_rerio.est_counts.masked.kallisto.tsv 
Input file found, reading: Gadus_morhua.est_counts.masked.kallisto.tsv 
Input file found, reading: Gallus_gallus.est_counts.masked.kallisto.tsv 
Input file found, reading: Homo_sapiens.est_counts.masked.kallisto.tsv 
Input file found, reading: Macaca_mulatta.est_counts.masked.kallisto.tsv 
Input file found, reading: Monodelphis_domestica.est_counts.masked.kallisto.tsv 
Input file found, reading: Mus_musculus.est_counts.masked.kallisto.tsv 
Input file found, reading: Ore

,Anolis_carolinensis_SRR391651,Anolis_carolinensis_SRR391653,Anolis_carolinensis_SRR5039728,Anolis_carolinensis_SRR5039729,Anolis_carolinensis_SRR5039730,Anolis_carolinensis_SRR5039731,Anolis_carolinensis_SRR5039732,Anolis_carolinensis_SRR5039733,Anolis_carolinensis_SRR5039734,Anolis_carolinensis_SRR5039735,⋯,Xenopus_tropicalis_SRR649393,Xenopus_tropicalis_SRR649394,Xenopus_tropicalis_SRR649395,Xenopus_tropicalis_SRR649396,Xenopus_tropicalis_SRR649397,Xenopus_tropicalis_SRR649398,Xenopus_tropicalis_SRR649399,Xenopus_tropicalis_SRR649400,Xenopus_tropicalis_SRR943352,Xenopus_tropicalis_SRR943353
OG0007342,203,210,110,321,47,44,79,111,96,31,⋯,146,193,278,273,69,66,551,279,1418,2288
OG0007346,601,608,634,1583,368,502,488,975,578,241,⋯,235,155,364,279,207,163,348,674,935,702
OG0007348,149,136,28,47,6,22,13,13,27,4,⋯,118,104,206,178,124,57,243,1296,1748,500
OG0007349,239,254,204,633,88,183,123,236,114,136,⋯,461,264,188,232,74,54,286,245,1071,914
OG0007350,192,204,70,436,44,79,136,48,83,46,⋯,1634,1784,1847,2287,1177,1416,1910,2044,1177,546
OG0007351,103,118,65,315,50,123,64,71,61,69,⋯,82,110,142,114,70,53,432,852,1548,744


In [5]:
df_nonzero = df_sog[,apply(df_sog, 2, sum)!=0]
cnf_in = edgeR::DGEList(counts = df_nonzero)

cnf_out1 = edgeR::calcNormFactors(cnf_in, method='TMM', refColumn=NULL)
x = cnf_out1[[2]][['norm.factors']]
cat('Round 1: Median normalization factor =', median(x), '\n')
median_value = sort(x)[ceiling(length(x)/2)]
median_index = (1:length(x))[x==median_value]

cnf_out2 = edgeR::calcNormFactors(cnf_in, method='TMM', refColumn=median_index)
cat('Round 2: Median normalization factor =', median(cnf_out2[[2]][['norm.factors']]), '\n')

df_nf = cnf_out2[[2]]
df_nf[['sample']] = rownames(df_nf)
df_nf = df_nf[,c('sample','group','lib.size','norm.factors')]
write.table(df_nf, 'normalization_factor.tsv', row.names=FALSE, sep='\t')


Round 1: Median normalization factor = 1.190107 
Round 2: Median normalization factor = 1.205314 


In [6]:
xlim = c(-2,2)
bins = seq(-2,2,0.1)
file_name='normalization_factor_histogram.pdf'
pdf(file_name, height=3.3, width=7.2) # full figure size = 9.7 x 7.2
x = log2(cnf_out2[[2]][['norm.factors']])
x[x>xlim[2]] = xlim[2]
x[x<xlim[1]] = xlim[1]
hist(x, xlab='log2(TMM normalization factor)', ylab='Count', main=NULL, col='black', xlim=xlim, breaks=bins, las=1)
graphics.off()

In [7]:
get_tmm_scaled_fpkm = function(dat, df_nf, efflen) {
    dat_fpkm = dat
    dat_fpkm[,] = NA
    for (sample in colnames(dat)) {
        if (sample %in% rownames(df_nf)) {
            dat_sample = data.frame(dat[,sample])
            colnames(dat_sample) = sample
            nf_sample = df_nf[sample,]
            dl_sample = list(counts=dat_sample, samples=nf_sample)
            class(dl_sample) = 'DGEList'
            col_fpkm = edgeR::rpkm(dl_sample, gene.length=efflen[,sample], normalized.lib.sizes=TRUE)
            dat_fpkm[,sample] = col_fpkm
        } else {
            dat_fpkm[,sample] = dat[,sample]
        }
    }
    return(dat_fpkm)
}

files = list.files(dir_count)
for (sp in names(uncorrected)) {
    infile = paste0(sp, '.est_counts.masked.kallisto.tsv')
    infile_path = paste0(dir_count, infile)
    if (file.exists(infile_path)) {
        cat('Input file found, reading:', infile, '\n')
        dat = read.table(infile_path, header=TRUE, row.names=1, sep='\t')
        dat = dat[,(colnames(dat)!='length')]
        df_nf = cnf_out2[[2]][startsWith(rownames(cnf_out2[[2]]),sp),]
        rownames(df_nf) = sub(paste0(sp,'_'),'',rownames(df_nf))
        path_efflen = paste0(dir_efflen, sp, '.eff_length.masked.kallisto.tsv')
        efflen = read.table(path_efflen, header=TRUE, row.names=1, sep='\t')
        dat_fpkm = get_tmm_scaled_fpkm(dat, df_nf, efflen)
        genus_cap = tolower(substring(sp,1,1))
        spec_name = strsplit(sp,'_')[[1]][2]
        idfile = paste0(dir_id, genus_cap, spec_name, '_gene_ensembl.tsv')
        dat_fpkm = transcript2gene_transformation(dat_fpkm, idfile)
        dat_fpkm = log2(dat_fpkm+1)
        file_fpkm = paste0(dir_tmm_logfpkm, sp, '.gene.log.tsv')
        write.table(dat_fpkm, file_fpkm, sep='\t', row.names=TRUE, col.names=TRUE, quote=FALSE)
    }
}

Input file found, reading: Anolis_carolinensis.est_counts.masked.kallisto.tsv 
Input file found, reading: Astyanax_mexicanus.est_counts.masked.kallisto.tsv 
Input file found, reading: Bos_taurus.est_counts.masked.kallisto.tsv 
Input file found, reading: Callithrix_jacchus.est_counts.masked.kallisto.tsv 
Input file found, reading: Canis_lupus.est_counts.masked.kallisto.tsv 
Input file found, reading: Chinchilla_lanigera.est_counts.masked.kallisto.tsv 
Input file found, reading: Danio_rerio.est_counts.masked.kallisto.tsv 
Input file found, reading: Gadus_morhua.est_counts.masked.kallisto.tsv 
Input file found, reading: Gallus_gallus.est_counts.masked.kallisto.tsv 
Input file found, reading: Homo_sapiens.est_counts.masked.kallisto.tsv 
Input file found, reading: Macaca_mulatta.est_counts.masked.kallisto.tsv 
Input file found, reading: Monodelphis_domestica.est_counts.masked.kallisto.tsv 
Input file found, reading: Mus_musculus.est_counts.masked.kallisto.tsv 
Input file found, reading: Ore

In [8]:
cat('Completed!\n')

Completed!
